# ReasoningAgnet (Tree of Thoughts with Beam Search)

## Introduction

This repository demonstrates how to implement a Tree of Thoughts (ToT) reasoning agent using beam search in under 50 lines of core code (excluding auxiliary components like visualization and logging). The `ReasoningAgent` leverages large language models (LLMs) such as GPT-4 or Llama to perform complex reasoning tasks by exploring multiple thought trajectories.

In the provided example, we showcase how the `ReasoningAgent`, even when using a smaller model (e.g., `gpt-4o-mini`), can effectively generate and evaluate multiple reasoning paths to solve intricate problems.

## What is the Tree of Thoughts and Beam Search?

The **Tree of Thoughts (ToT)** is a reasoning framework where an agent considers multiple possible reasoning steps (thoughts) and organizes them into a tree structure. **Beam search** is an optimization algorithm that explores a subset of the tree by keeping the top `k` options at each level (the beam size). Combining ToT with beam search allows the agent to efficiently navigate through the most promising thought trajectories.

Use **Beam Size = 1** for a special case of O1-style reasoning, which is similar to CoT with multiple prompts. 


## Key Features

- **Enhanced Problem-Solving**: Implements the Tree of Thoughts framework to improve reasoning capabilities.
- **Beam Search Optimization**: Utilizes beam search to efficiently explore and evaluate multiple thought paths.
- **Multi-Agent Collaboration**: Includes a thinker agent to generate possible next steps and a grader agent to evaluate them.
- **Visualization Support**: Provides visualization of the thought tree using Graphviz for better analysis.
- **Customizable Parameters**: Allows configuration of maximum depth, beam size, and answer selection approach.

In [1]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

config_list = [{"model": "gpt-4o-mini", "api_key": api_key}]
verbose = False

## Simple Chain-of-Thought O1-style 

In [2]:
from autogen import AssistantAgent, UserProxyAgent
from autogen.agentchat.contrib.reasoning_agent import ReasoningAgent, ThinkNode, visualize_tree

reason_agent = ReasoningAgent(
    name="reason_agent",
    system_message="answer math questions",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=1,  # using beam size 1 for special case of O1-style reasoning.
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config=False,
    max_consecutive_auto_reply=10,
)

In [3]:
question = "What is the expected maximum dice value if you can roll a 6-sided dice three times?"


def last_meaningful_msg(sender, recipient, summary_args):
    import warnings

    if sender == recipient:
        return "TERMINATE"

    summary = ""
    chat_messages = recipient.chat_messages[sender]

    for msg in reversed(chat_messages):
        try:
            content = msg["content"]
            if isinstance(content, str):
                summary = content.replace("TERMINATE", "")
            elif isinstance(content, list):
                # Remove the `TERMINATE` word in the content list.
                summary = "\n".join(
                    x["text"].replace("TERMINATE", "") for x in content if isinstance(x, dict) and "text" in x
                )
            if summary.strip().rstrip():
                return summary
        except (IndexError, AttributeError) as e:
            warnings.warn(f"Cannot extract summary using last_msg: {e}. Using an empty str as summary.", UserWarning)
    return summary

In [4]:
ans = user_proxy.initiate_chat(reason_agent, message=question, summary_method=last_meaningful_msg)

user_proxy (to reason_agent):

What is the expected maximum dice value if you can roll a 6-sided dice three times?

--------------------------------------------------------------------------------
reason_agent (to tot_thinker):

# Question: What is the expected maximum dice value if you can roll a 6-sided dice three times?
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to reason_agent):

**Reflection**  
The previous steps do not include any explicit actions or calculations related to the question posed. The user is seeking a mathematical expectation regarding a 6-sided die rolled three times, and there is no previous trajectory to assess mistakes in. The handling of probabilities and expectations is essential to derive the correct answer, which seems to be missing.

**Possible Options:**  
Option 1: Calculate the expected value for one die roll and then find the maximum value for three rolls.  
Optio

In [5]:
print(ans.summary)

To calculate the expected maximum value when rolling a 6-sided die three times, we can follow these steps:

### Step 1: Breakdown of the Calculation

The maximum value from three rolls of a die can take on values from 1 to 6. To find the expected maximum value, we need to calculate the probability of the maximum value being each possible outcome (1, 2, 3, 4, 5, or 6) and then sum those probabilities multiplied by their corresponding values.

1. **Determine the probability of the maximum value being k (for k = 1 to 6)**:
   - The maximum will be less than or equal to \( k \) if all three rolls are \( \leq k \).
   - The probability of one die rolling \( \leq k \) is \( \frac{k}{6} \).
   - Thus, the probability that all three rolls (i.e., the maximum) are \( \leq k \) is \( \left(\frac{k}{6}\right)^3 \).
   - Therefore, the probability that the maximum is exactly \( k \) is:
     \[
     P(X = k) = P(X \leq k) - P(X \leq k - 1) = \left(\frac{k}{6}\right)^3 - \left(\frac{k-1}{6}\right)^3

## Larger Beam Size Search in Tree of Thought

In [6]:
reason_agent = ReasoningAgent(
    name="reason_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=3,
    max_depth=3,
)
user_proxy = UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
    max_consecutive_auto_reply=10,
)

In [7]:
ans = user_proxy.initiate_chat(
    reason_agent,
    message="Design a mixed integer linear program for a coffee roasting supply chain",
    summary_method=last_meaningful_msg,
)

user_proxy (to reason_agent):

Design a mixed integer linear program for a coffee roasting supply chain

--------------------------------------------------------------------------------
reason_agent (to tot_thinker):

# Question: Design a mixed integer linear program for a coffee roasting supply chain
---
What are the possible next steps?

--------------------------------------------------------------------------------
tot_thinker (to reason_agent):

**Reflection**
The previous steps did not provide any specific details regarding the formulation of the mixed integer linear program (MILP) for the coffee roasting supply chain. It's essential to include constraints, decision variables, and objective functions in a structured manner. There's a need to analyze the key components that impact the supply chain effectively.

**Possible Options:**
Option 1: Identify and define the decision variables for the MILP, such as the amounts of coffee beans to purchase, roast, and distribute.
Option 2: O

In [8]:
print(ans.summary)

To design a mixed integer linear program (MILP) for a coffee roasting supply chain, we will follow the structured approach outlined in the provided possibilities. Here’s how we can approach it:

### Step 1: Outline the Objective Function
We need to define an objective function that optimizes costs. Let’s consider the components that could influence costs in a coffee roasting supply chain. The objective could be expressed as minimizing total costs, which could include:

- **Transportation Costs (T)**: Cost to transport raw beans from suppliers to roasting facilities and finished products to distribution centers.
- **Roasting Costs (R)**: Costs associated with roasting the coffee, which may depend on the quantity of coffee roast.
- **Storage Costs (S)**: Costs to store raw coffee beans and finished products.

The overall objective function could be formulated as:

\[ \text{Minimize } Z = T + R + S \]

### Step 2: Identify Constraints
Next, we should identify the constraints that would af

In [9]:
!pip install graphviz

In [10]:
visualize_tree(reason_agent._root)

![](tree_of_thoughts.png)

### ReasoningAgent with Nested Chats

In [11]:
writer = AssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)
reason_agent_for_writer = ReasoningAgent(
    name="reason_agent",
    llm_config={"config_list": config_list},
    verbose=verbose,
    beam_size=1,
    max_depth=3,
)


def reflection_message(recipient, messages, sender, config):
    print("Reflecting...", "yellow")
    return f"Reflect, Reason and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

In [12]:
user_proxy.register_nested_chats(
    [
        {
            "recipient": reason_agent_for_writer,
            "message": reflection_message,
            "summary_method": "last_msg",
            "max_turns": 1,
        }
    ],
    trigger=writer,
)

In [13]:
task = """Write a concise but engaging blogpost about Nvida."""
res = user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

user_proxy (to Writer):

Write a concise but engaging blogpost about Nvida.

--------------------------------------------------------------------------------
Writer (to user_proxy):

### The Rise of NVIDIA: Powering the Future of Technology

In the world of technology, few companies have managed to redefine an industry like NVIDIA. Originally founded in 1993, NVIDIA carved its niche by revolutionizing graphics processing units (GPUs), essential for rendering stunning visuals in video games. However, as the digital landscape has evolved, so has NVIDIA, positioning itself at the forefront of groundbreaking innovations in artificial intelligence (AI), self-driving cars, and high-performance computing.

#### The Graphics Giant

At its core, NVIDIA’s strength remains in its GPUs. With the launch of the GeForce series, the company became synonymous with high-quality gaming, providing enthusiasts with unparalleled graphics and performance. But NVIDIA didn’t stop there. As gaming turned into a

## Save data to future training

In [14]:
import json
import pickle

In [15]:
def extract_sft_dataset(root):
    """
    Extract the best trajectory or multiple equally good trajectories
    for SFT training.

    Args:
        root: The root node of the tree.

    Returns:
        List of best trajectories, where each trajectory is a pair of instruction and response.
    """
    instruction = root.content
    idx = len("# Question: ") + len(root.content) + 1

    def find_leaf_nodes(node):
        """Recursively find all leaf nodes."""
        if not node.children:
            return [node]
        leafs = []
        for child in node.children:
            leafs.extend(find_leaf_nodes(child))
        return leafs

    # Step 1: Find all leaf nodes
    leaf_nodes = find_leaf_nodes(root)

    # Step 2: Determine the highest score among leaf nodes
    max_value = max(leaf_nodes, key=lambda x: x.value).value

    # Step 3: Collect all leaf nodes with the highest score
    best_leafs = [leaf for leaf in leaf_nodes if leaf.value == max_value]

    # Step 4: Collect trajectories for all the best leaf nodes
    best_trajectories = [{"instruction": instruction, "response": leaf.trajectory[idx:]} for leaf in best_leafs]

    return best_trajectories


# Example usage
sft_data = extract_sft_dataset(reason_agent._root)
print(json.dumps(sft_data, indent=2)[:1000], "... skip details ...")

[
  {
    "instruction": "Design a mixed integer linear program for a coffee roasting supply chain",
    "response": "Step 1: Identify and define the decision variables for the MILP, such as the amounts of coffee beans to purchase, roast, and distribute.\nStep 2: Define constraints related to the supply chain, such as capacity limits, demand requirements, and roasting schedules."
  },
  {
    "instruction": "Design a mixed integer linear program for a coffee roasting supply chain",
    "response": "Step 1: Identify and define the decision variables for the MILP, such as the amounts of coffee beans to purchase, roast, and distribute.\nStep 2: Formulate the objective function for the MILP, which could focus on minimizing costs or maximizing profit."
  },
  {
    "instruction": "Design a mixed integer linear program for a coffee roasting supply chain",
    "response": "Step 1: Identify and define the decision variables for the MILP, such as the amounts of coffee beans to purchase, roast, 

In [16]:
def extract_rlhf_preference_dataset(root, contrastive_threshold=0.2):
    """
    Extract and generate preference pairs for RLHF training by comparing sibling nodes.

    Args:
        root: The root node of the tree.
        contrastive_threshold (float): between (0, 1), a distance measure that we are confidence to call
            one is positive and another is negative.

    Returns:
        A list of preference pairs, where each pair contains two responses and
        indicates which one is preferred.
    """
    preference_pairs = []

    assert contrastive_threshold > 0
    assert contrastive_threshold < 1

    def traverse_tree(node):
        """Traverse the tree to compare sibling nodes and collect preferences."""
        if not node.children:
            return  # Leaf node, no comparisons needed

        # Step 1: Compare all sibling nodes
        for i in range(len(node.children)):
            for j in range(len(node.children)):
                if i == j:
                    continue
                child_a, child_b = node.children[i], node.children[j]
                if child_a.value - child_b.value > contrastive_threshold:
                    preference_pairs.append(
                        {
                            "instruction": node.trajectory,
                            "preferred_response": f"Step {child_a.depth}: {child_a.content}",
                            "dispreferred_response": f"Step {child_b.depth}: {child_b.content}",
                        }
                    )

        # Step 2: Recurse into child nodes
        for child in node.children:
            traverse_tree(child)

    # Start traversal from the root
    traverse_tree(root)

    return preference_pairs


# Example usage
rlhf_data = extract_rlhf_preference_dataset(reason_agent._root)

print(f"There are {len(rlhf_data)} pairs of data\n\n")
print(json.dumps(rlhf_data, indent=2)[:2000], "... skip details ...")

There are 17 pairs of data


[
  {
    "instruction": "# Question: Design a mixed integer linear program for a coffee roasting supply chain",
    "preferred_response": "Step 1: Outline the objective function that optimizes costs, profit, or delivery times in the coffee roasting supply chain context.",
    "dispreferred_response": "Step 1: Identify and define the decision variables for the MILP, such as the amounts of coffee beans to purchase, roast, and distribute."
  },
  {
    "instruction": "# Question: Design a mixed integer linear program for a coffee roasting supply chain",
    "preferred_response": "Step 1: Outline the objective function that optimizes costs, profit, or delivery times in the coffee roasting supply chain context.",
    "dispreferred_response": "Step 1: Develop a visual representation of the supply chain process which can aid in understanding the interactions among different components before finalizing the MILP."
  },
  {
    "instruction": "# Question: Design a 

In [17]:
data = reason_agent._root.to_dict()
with open("reasoning_tree.json", "w") as f:
    json.dump(data, f)

# recover the node
new_node = ThinkNode.from_dict(json.load(open("reasoning_tree.json", "r")))